In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Sequential(nn.Conv2d(1, 8, 3, padding=0),nn.ReLU(inplace=True),nn.Dropout(p=0.1),nn.BatchNorm2d(8)) #input -28x28x1 OUtput - 26x26x8 
        self.conv2 = nn.Sequential(nn.Conv2d(8, 16, 3, padding=0),nn.ReLU(inplace=True),nn.Dropout(p=0.1),nn.BatchNorm2d(16))   #input -26x26x8 OUtput - 24x24x16 
        self.conv3 = nn.Sequential(nn.Conv2d(16, 32, 3, padding=0),nn.ReLU(inplace=True),nn.Dropout(p=0.1),nn.BatchNorm2d(32))  #input -24x24x16 OUtput - 22x22x32 


        self.conv4 = nn.Sequential(nn.Conv2d(32, 8, 1, padding=0),nn.ReLU(inplace=True),nn.BatchNorm2d(8)) #input -22x22x32 OUtput - 22x22x8 
        self.pool1 = nn.Sequential(nn.MaxPool2d(kernel_size=2, stride=2))  #input - 22x22x8 OUtput - 11x11x8 

        self.conv5 = nn.Sequential(nn.Conv2d(8, 16, 3, padding=0),nn.ReLU(inplace=True),nn.Dropout(p=0.1),nn.BatchNorm2d(16))  #input -11x11x8 OUtput - 9x19x16 

        self.conv6 = nn.Sequential(nn.Conv2d(16, 8, 1, padding=0),nn.ReLU(inplace=True),nn.BatchNorm2d(8)) #input -9x9x16 OUtput - 9x9x8 

        self.conv7 = nn.Sequential(nn.Conv2d(8, 16, 3, padding=0),nn.ReLU(inplace=True),nn.Dropout(p=0.1),nn.BatchNorm2d(16))   #input -9x9x8 OUtput - 7x7x16 
        
        self.conv8 = nn.Sequential(nn.Conv2d(16, 10, 7))   #input -7x7x16 OUtput - 1x1x10 

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool1(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)        
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [32]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
              ReLU-2            [-1, 8, 26, 26]               0
           Dropout-3            [-1, 8, 26, 26]               0
       BatchNorm2d-4            [-1, 8, 26, 26]              16
            Conv2d-5           [-1, 16, 24, 24]           1,168
              ReLU-6           [-1, 16, 24, 24]               0
           Dropout-7           [-1, 16, 24, 24]               0
       BatchNorm2d-8           [-1, 16, 24, 24]              32
            Conv2d-9           [-1, 32, 22, 22]           4,640
             ReLU-10           [-1, 32, 22, 22]               0
          Dropout-11           [-1, 32, 22, 22]               0
      BatchNorm2d-12           [-1, 32, 22, 22]              64
           Conv2d-13            [-1, 8, 22, 22]             264
             ReLU-14            [-1, 8,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [35]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 19
for epoch in range(EPOCHS):
    print("EPOCH : ",epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

EPOCH :  0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0558, Accuracy: 9828/10000 (98.28%)

EPOCH :  1


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0222, Accuracy: 9929/10000 (99.29%)

EPOCH :  17


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0204, Accuracy: 9935/10000 (99.35%)

EPOCH :  18


loss=0.028398161754012108 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 27.50it/s]



Test set: Average loss: 0.0202, Accuracy: 9942/10000 (99.42%)

